In [1]:
import pandas as pd
import torch
from torch import nn
from transformers import BertTokenizer, BertModel

In [3]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("dbmdz/bert-base-turkish-cased")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 4)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask, return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

model_offensive = torch.load('../../../models_notebooks/transformers/models/bert_model_is_offensive.pt')
model_target = torch.load('../../../models_notebooks/transformers/models/bert_model_target.pt')
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
mapping = {0: 'INSULT', 1: 'RACIST', 2: 'SEXIST', 3: 'PROFANITY'}

def predict_single(model, sentence):
        use_cuda = torch.cuda.is_available()
        device = torch.device("cuda" if use_cuda else "cpu")
        if use_cuda:
            model = model.cuda()
        with torch.no_grad():
            input_id = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
            mask = input_id['attention_mask'].to(device)
            input_id = input_id['input_ids'].squeeze(1).to(device)
            output = model(input_id, mask)
            return output.argmax(dim=1).item()

In [7]:
df = pd.read_csv('../../train_clean.csv')
df["is_offensive"] = 0
df.label.value_counts()

0    22589
1    19809
Name: label, dtype: int64

In [11]:
df = df[df['label'] == 0]
df.reset_index(drop=True, inplace=True)
df

,id,text,label,is_offensive
0,26418.0,gerçekten sizin hikayelerinizi izleyerek mi ye...,0,0
1,14473.0,çoook çok bi baklava bi sen zaten,0,1
2,16107.0,sn dükel atatürkün karma e m ile başlattığı s...,0,0
3,45908.0,konfederasyonumuzun aile ve sosyal politikalar...,0,0
4,22887.0,emeklilikte yaşa takılanlar perişan ç...,0,0
...,...,...,...,...
22584,13757.0,seni nasıl mutlu edelim sen her zaman mutlusu...,0,0
22585,25088.0,babacim babacim öztürk serengil izlemek istiyo...,0,0
22586,34517.0,bir kesinlik üzerine konuşmuyorum tabiki de ...,0,0
22587,30141.0,nap of a star tam böyle hıçkıra hıçkıra ağlama...,0,0


In [12]:
for i in range(len(df)):
    df['is_offensive'][i] = predict_single(model_offensive, df['text'][i])

C:\Users\hasan\AppData\Local\Temp/ipykernel_1044/480479499.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_offensive'][i] = predict_single(model_offensive, df['text'][i])


In [13]:
not_offensive = df[(df['is_offensive'] == 0) & (df['label'] == 0)]
not_offensive

,id,text,label,is_offensive
0,26418.0,gerçekten sizin hikayelerinizi izleyerek mi ye...,0,0
2,16107.0,sn dükel atatürkün karma e m ile başlattığı s...,0,0
3,45908.0,konfederasyonumuzun aile ve sosyal politikalar...,0,0
4,22887.0,emeklilikte yaşa takılanlar perişan ç...,0,0
5,42028.0,kykyurtzammınahayır öğrenci sizin gelir kaynag...,0,0
...,...,...,...,...
22584,13757.0,seni nasıl mutlu edelim sen her zaman mutlusu...,0,0
22585,25088.0,babacim babacim öztürk serengil izlemek istiyo...,0,0
22586,34517.0,bir kesinlik üzerine konuşmuyorum tabiki de ...,0,0
22587,30141.0,nap of a star tam böyle hıçkıra hıçkıra ağlama...,0,0


In [15]:
# take a sample of 2000 rows randomly
not_offensive = not_offensive.sample(n=2000, random_state=42)
not_offensive.reset_index(drop=True, inplace=True)
not_offensive.drop(columns=['label', 'id'], inplace=True)
not_offensive

,text,is_offensive
0,varant yarın hem varantların hemde benim vade ...,0
1,onları adam yerine koymimben kimseye borsacı o...,0
2,ali bey dm ye gelin,0
3,seks seks seks dikkatinizi çektiğime göre...,0
4,reynmenin klibini muhsin yazıcıoğlunun anısına...,0
...,...,...
1995,ve taşlar yerinden oynamadan savaş biter kaz...,0
1996,şirketin iş kendi hissesine sahip çıkıp bika...,0
1997,telefonum kapandı acılmıyor bugun sinir krizi ...,0
1998,binali bey lütfen hazda oyunuzu ekrem imamoğ...,0


In [21]:
not_offensive.text[3]

'seks seks seks      dikkatinizi çektiğime göre bu akşamlık blutv üyeliği paylaşabilecek var mı    '

In [21]:
offensive = df[(df['is_offensive'] == 1) & (df['label'] == 1)]
offensive

,id,text,label,is_offensive
4,12878.0,hakemler tarih yazıyorlar kişiye karşı önde...,1,1
12,NaN,liste hazırlayan kız senin şimdi ebeni sikmeye...,1,1
19,NaN,bide suriyeli oç httpstcoaurmtqme,1,1
20,NaN,arap özentiliğini bırakamadınız amk şeriatçıları,1,1
36,NaN,hocam zor sorunca fazla maaş mi alıyorsunuz ac...,1,1
...,...,...,...,...
42388,NaN,kum döken mallar varonlara ayrı uyuz oldum am...,1,1
42390,NaN,yarın kendime bi bakım yapıp foto atayımda erk...,1,1
42392,NaN,cryptosfun şimdi orospu çocuğuyum diye anır am...,1,1
42395,NaN,prodbyjaysou kkesbekes ya kaybol twitimden kod...,1,1


In [25]:
for i in range(len(offensive)):
    offensive.iloc[i, 3] = mapping[predict_single(model_target, offensive.iloc[i, 1])]

c:\Users\hasan\anaconda3\envs\DataScience\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [30]:
offensive.columns = ['id', 'text', 'label', 'target']
offensive

,id,text,label,target
4,12878.0,hakemler tarih yazıyorlar kişiye karşı önde...,1,RACIST
12,NaN,liste hazırlayan kız senin şimdi ebeni sikmeye...,1,PROFANITY
19,NaN,bide suriyeli oç httpstcoaurmtqme,1,RACIST
20,NaN,arap özentiliğini bırakamadınız amk şeriatçıları,1,RACIST
36,NaN,hocam zor sorunca fazla maaş mi alıyorsunuz ac...,1,RACIST
...,...,...,...,...
42388,NaN,kum döken mallar varonlara ayrı uyuz oldum am...,1,PROFANITY
42390,NaN,yarın kendime bi bakım yapıp foto atayımda erk...,1,SEXIST
42392,NaN,cryptosfun şimdi orospu çocuğuyum diye anır am...,1,PROFANITY
42395,NaN,prodbyjaysou kkesbekes ya kaybol twitimden kod...,1,PROFANITY


In [32]:
df[(df['label'] == 0) & (df['is_offensive'] == 1)]

,id,text,label,is_offensive
1,14473.0,çoook çok bi baklava bi sen zaten,0,1
8,21684.0,toprağın üstünde aziz ettin toprağın altınd...,0,1
11,41982.0,kimseye tahammül edemedigim evredeyim yine all...,0,1
21,16364.0,yok yok olmaz böyle bi şey,0,1
26,22263.0,kabristan diye okudum allah affetsin gerçi p...,0,1
...,...,...,...,...
32493,30181.0,yoğun yorucu hayatın içinde gülüşüne bakıp din...,0,1
32506,27055.0,sokakta yatan insanlar ve sokak hayvanlarının ...,0,1
32510,34722.0,valla ben fenerli olduğunu bile bilmiyordum b...,0,1
32511,19398.0,konforsuz inhidam twitarttircomtakipçikazan ya...,0,1


In [34]:
not_offensive.to_csv('./new_generated/kaggle_zehra/not_offensive.csv', index=False)
offensive.to_csv('./new_generated/kaggle_zehra/offensive.csv', index=False)